In [2]:
import numpy as np
import pandas as pd

In [3]:
def str2flt(str_list):
    flt_list=[]
    for i in str_list:
        flt_list.append(float(i))
    return flt_list

def dis_calc(atom_co_1,atom_co_2):
    dis=np.sqrt((atom_co_1[0]-atom_co_2[0])**2+(atom_co_1[1]-atom_co_2[1])**2+(atom_co_1[2]-atom_co_2[2])**2)
    return dis

In [4]:
def atom_info_grep(data_file_name):
    atom_info_list=[]
    C_i, H_i = 0, 0
    C_cat_i, S_cat_i, Mg_cat_i = 0, 0, 0
    H_h2_i = 0
    with open(data_file_name,'r') as f:
        for line in f.readlines()[2:]:
            line_content=line.split()
            atom_type=line_content[0]
            if atom_type=='C':
                atome_name=atom_type+'_'+str(C_i)
                atom_info_list.append([atome_name,atom_type]+line_content[1:])
                C_i+=1
            elif atom_type=='H':
                atome_name=atom_type+'_'+str(H_i)
                atom_info_list.append([atome_name,atom_type]+line_content[1:])
                H_i+=1
            elif atom_type=='C_cat':
                atome_name=atom_type+'_'+str(C_cat_i)
                atom_info_list.append([atome_name,atom_type]+line_content[1:])
                C_cat_i+=1
            elif atom_type=='O_cat':
                atome_name=atom_type+'_'+str(S_cat_i)
                atom_info_list.append([atome_name,atom_type]+line_content[1:])
                S_cat_i+=1
            elif atom_type=='Mg_cat':
                atome_name=atom_type+'_'+str(Mg_cat_i)
                atom_info_list.append([atome_name,atom_type]+line_content[1:])
                Mg_cat_i+=1
            elif atom_type=='H_h2':
                atome_name=atom_type+'_'+str(H_h2_i)
                atom_info_list.append([atome_name,atom_type]+line_content[1:])
                H_h2_i+=1
    atom_info_df = pd.DataFrame(atom_info_list,columns=['atom_name','atom_type','x','y','z'])
    atom_info_df = add_H_com(atom_info_df)
    return atom_info_df

def add_H_com(atom_info_df):
    H_1_co=str2flt(atom_info_df[atom_info_df.atom_name=='H_h2_0'].values[0,2:])
    H_2_co=str2flt(atom_info_df[atom_info_df.atom_name=='H_h2_1'].values[0,2:])
    H_com_co=[]
    for i in range(3): H_com_co.append((H_1_co[i]+H_2_co[i])/2)
    H_com_info=pd.Series(['H_com_0','H_com']+H_com_co,index=atom_info_df.columns)
    atom_info_df=pd.concat([atom_info_df,H_com_info],ignore_index=True)
    return atom_info_df

def H2_atom_info_grep(atom_info_df):
    H_1_co=str2flt(atom_info_df[atom_info_df.atom_name=='H_h2_0'].values[0,2:])
    H_2_co=str2flt(atom_info_df[atom_info_df.atom_name=='H_h2_1'].values[0,2:])
    H_com_co=str2flt(atom_info_df[atom_info_df.atom_name=='H_com_0'].values[0,2:])
    return H_1_co,H_2_co,H_com_co
    
def atom_distance_calc(atom_info_df):
    H_1_co,H_2_co,H_com_co=H2_atom_info_grep(atom_info_df)
    atom_distance_list=[]
    for atom_info_line in atom_info_df.iloc[:-3,:].values:
        atom_co=str2flt(atom_info_line[2:])
        dis_1=dis_calc(H_1_co,atom_co)
        dis_2=dis_calc(H_2_co,atom_co)
        dis_3=dis_calc(H_com_co,atom_co)
        atom_name_1=atom_info_line[0]
        atom_type_1=atom_info_line[1]
        atom_distance_list.append([atom_name_1,'H_h2_0',atom_type_1,'H_h2',dis_1])
        atom_distance_list.append([atom_name_1,'H_h2_1',atom_type_1,'H_h2',dis_2])
        atom_distance_list.append([atom_name_1,'H_com_0',atom_type_1,'H_com',dis_3])
    atom_distance_df=pd.DataFrame(atom_distance_list,
                                  columns=['atom_name_1','atom_name_2','atom_type_1','atom_type_2','distance'])
    return atom_distance_df

In [5]:
# Calc the LJ-coulomb energy of each pair of atoms
def LJ_coulomb_energy_calc(atom1_type,atom2_type,r):
    atom_1_FF_info=LJ_FF_coulomb_params[LJ_FF_coulomb_params.atom_type==atom1_type]
    atom_2_FF_info=LJ_FF_coulomb_params[LJ_FF_coulomb_params.atom_type==atom2_type]
    if atom1_type != 'O_cat' and atom1_type != 'Mg_cat' and atom1_type != 'C_cat':
        LJ_eps_1 = atom_1_FF_info['epsilon/kB'].values[0]
        LJ_eps_2 = atom_2_FF_info['epsilon/kB'].values[0]
        LJ_sig_1 = atom_1_FF_info['sigma'].values[0]
        LJ_sig_2 = atom_2_FF_info['sigma'].values[0]
        m_1 = atom_1_FF_info['mass'].values[0]
        m_2 = atom_2_FF_info['mass'].values[0]
        m_r = (m_1 * m_2) / (m_1 + m_2)
        #Lorentz-Berthelot rules
        LJ_eps = np.sqrt(LJ_eps_1*LJ_eps_2)
        LJ_sig = (LJ_sig_1+LJ_sig_2)/2
        # LJ potential energy calc
        E_LJ = 4 * LJ_eps * ((LJ_sig/r)**12 - (LJ_sig/r)**6)
        # LJ-FH corrected potential energy calc
        E_FH = ((h_bar **2)/(24*m_r*kB*T))*4*LJ_eps*(132*(LJ_sig/r)**12-30*(LJ_sig/r)**6)/r**2
        E_LJ += E_FH
    else: E_LJ = 0
    # Coulomb potential energy calc
    q_1 = atom_1_FF_info['q_'+site_type].values[0]
    q_2 = atom_2_FF_info['q_'+site_type].values[0]
    E_coulomb = ke * (q_1 * q_2) / r
    return E_LJ  + E_coulomb

# Calc the LJ-coulomb energy of whole system
def system_LJ_energy_calc(atom_distance_info):
    E_LJ_sum=0
    for line in atom_distance_info.values:
        atom1_type = line[2]
        atom2_type = line[3]
        r = line[4]
        if r > cutoff: continue
        else:
            E_LJ = LJ_coulomb_energy_calc(atom1_type,atom2_type,r)
            E_LJ_sum+=E_LJ
    return E_LJ_sum

In [7]:
# kJ/mol
h_bar=1.054572e-34 * 6.62607015e-34 #internal units
kB=0.008314511
ke = 1389.354
cutoff = 12.8

LJ_FF_coulomb_params=pd.read_csv('FF_LJ_coulomb_params.csv')
# xyz_file_E_binding=pd.read_csv('xyz_file_binding_energy.csv')
# xyz_file_list=xyz_file_E_binding.struc_name.values.tolist()
xyz_FILE_DIR='data/'

In [8]:
#calculate E_LJ in advanced
E_LJ_list=[]
for i,struc_name in enumerate(xyz_file_list):
    type_index=struc_name.split('_')[0][-1]
    conj_index=struc_name.split('_')[-1]
    E_LJ_line=[struc_name]
    for site_type in ['single','two']:
        E_MP2=xyz_file_E_binding['E_binding_'+site_type][i]
        xyz_file_pwd=xyz_FILE_DIR+site_type+'_site/type'+str(type_index)+'/type'+str(type_index)+'_'+str(conj_index)+'.xyz'
        atom_distance_info=atom_distance_calc(atom_info_grep(xyz_file_pwd))
        E_LJ=system_LJ_energy_calc(atom_distance_info)
        E_LJ_line.append(E_LJ)
    E_LJ_list.append(E_LJ_line)
pd.DataFrame(E_LJ_list,columns=['struc_name','E_LJ_single','E_LJ_two']).to_csv('E_LJ.csv',index=False)

In [11]:
#calculate E_LJ in advanced
# xyz_FILE_DIR_all='F:/H2_COF_proj/each_steps_xyz_files/'
for T in [111,231,296]:
    E_LJ_list=[]
    for type_index in np.arange(0,9):
        for conj_index in np.arange(15,61):
            struc_name = 'type'+str(type_index)+'_'+str(conj_index)
            E_LJ_line=[struc_name]
            for site_type in ['single','two']:
    #             E_MP2=xyz_file_E_binding['E_binding_'+site_type][i]
                xyz_file_pwd=xyz_FILE_DIR+site_type+'_site/type'+str(type_index)+'/type'+str(type_index)+'_'+str(conj_index)+'.xyz'
                atom_distance_info=atom_distance_calc(atom_info_grep(xyz_file_pwd))
                E_LJ=system_LJ_energy_calc(atom_distance_info)
    #             print("E_LJ:",E_LJ)
                E_LJ_line.append(E_LJ)
            E_LJ_list.append(E_LJ_line)
    pd.DataFrame(E_LJ_list,columns=['struc_name','E_LJ_single','E_LJ_two']).to_csv('E_LJ_all_FH_'+str(T)+'.csv',index=False)

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

...

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The

/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  atom_info_df=atom_info_df.append(H_com_info,ignore_index=True)
/tmp/ipykernel_1223/4258613881.py:44: FutureWarning: The